### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import random
from PIL import Image
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# Colabのローカルファイルに自作データをコピー（初めからローカルファイルにアップロードする場合は不要）
!cp -r "/content/drive/MyDrive/HandSigns/" "/content/HandSigns/"

In [5]:
# 自作データをリサイズして、ファイル構造を保ったまま新しいフォルダに保存する関数を定義
def resize_images(base_dir, new_base_dir, image_size=(32, 32)):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        new_folder_path = os.path.join(new_base_dir, folder)
        # リサイズした自作データを保存する新しいフォルダを作成
        os.makedirs(new_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 拡張子の確認
            if file_path.endswith(('.jpg', '.jpeg')):
                # 画像を展開
                img = Image.open(file_path)
                # 画像をリサイズ
                img_resized = img.resize(image_size)
                # リサイズした画像を新しいフォルダに保存
                img_resized.save(os.path.join(new_folder_path, filename))

# 学習用の自作データのディレクトリを設定
train_base_dir = '/content/HandSigns/train'
# テスト用の自作データのディレクトリを設定
test_base_dir = '/content/HandSigns/test'

# リサイズした学習用の自作データを保存するディレクトリを設定
new_train_base_dir = '/content/HandSigns/resized_train_32'
# リサイズしたテスト用の自作データを保存するディレクトリを設定
new_test_base_dir = '/content/HandSigns/resized_test_32'
# リサイズする大きさを設定
image_size = (32, 32)

# 学習用の自作データをリサイズ
resize_images(train_base_dir, new_train_base_dir, image_size)
# テスト用の自作データをリサイズ
resize_images(test_base_dir, new_test_base_dir, image_size)

In [6]:
# 平均値と標準偏差を計算するためのHandSignsの前処理を定義
pre_transform_handsigns = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのHandSignsデータセットの前処理を定義
pre_train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=pre_transform_handsigns)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_handsigns = 0.0
pre_std_handsigns = 0.0
pre_total_samples_handsigns = len(pre_train_dataset_handsigns)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_handsigns:
    pre_image, _ = data
    pre_mean_handsigns += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_handsigns += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_handsigns /= pre_total_samples_handsigns
pre_std_handsigns /= pre_total_samples_handsigns

print("データセット全体の平均値: ", pre_mean_handsigns)
print("データセット全体の標準偏差: ", pre_std_handsigns)

データセット全体の平均値:  tensor([0.6954, 0.6754, 0.6450])
データセット全体の標準偏差:  tensor([0.1331, 0.1565, 0.1689])


In [7]:
# 学習用のHandSignsデータセットの前処理を定義
train_transform_handsigns = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# テスト用のHandSignsデータセットの前処理を定義
test_transform_handsigns = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# 学習用のHandSignsデータセットの読み込み
train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=train_transform_handsigns)
# テスト用のHandSignsデータセットの読み込み
test_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_test_32", transform=test_transform_handsigns)

# 学習用のHandSignsデータローダーを作成
train_loader_handsigns = torch.utils.data.DataLoader(dataset=train_dataset_handsigns, batch_size=16, shuffle=True, num_workers=2)
# テスト用のHandSignsデータローダーを作成
test_loader_handsigns = torch.utils.data.DataLoader(dataset=test_dataset_handsigns, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# 学習データのクラス名とラベルの対応を確認
print(train_dataset_handsigns.class_to_idx)
# テストデータのクラス名とラベルの対応を確認
print(test_dataset_handsigns.class_to_idx)

{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}
{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}


In [9]:
# イテレータを作成
data_iter = iter(train_loader_handsigns)

# データを1バッチ分取得
images, labels = next(data_iter)

# 取得したバッチの画像とラベルの情報を確認
print("画像の形状:", images.shape)  # (バッチサイズ, チャンネル数, 高さ, 幅)
print("ラベル:", labels)

画像の形状: torch.Size([16, 3, 32, 32])
ラベル: tensor([3, 7, 4, 1, 3, 5, 6, 9, 9, 5, 3, 0, 4, 0, 2, 1])


### モデルの実装

In [10]:
# 1×1の畳み込みを定義
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3の畳み込みを定義
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [11]:
# Residual Blocksを定義
class BasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_channels, channels, stride)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(channels, channels)
        self.bn2 = nn.BatchNorm2d(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [12]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Heの初期化（正規分布）
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                # 重みを1に初期化
                nn.init.constant_(m.weight, 1)
                # バイアスを0に初期化
                nn.init.constant_(m.bias, 0)

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [13]:
# モデルを呼び出す関数を定義
def resnet34_10():
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=10)

In [ ]:
# パラメータ数を計算する関数を定義
def num_params(model):
    return sum(x.numel() for x in model.parameters() if x.requires_grad)
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)
# モデルのパラメータ数を確認
print("ResNet34のパラメータ数: {}".format(num_params(model)))

ResNet34のパラメータ数: 21289802


### 学習と評価（10 epochs）

In [30]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 10
# warm-upするepoch数を設定
warmup_epochs = 1

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet34_HandSigns_10epochs_CLRS_restest.pth')

Epoch [1/10], Train Accuracy: 11.0 %, Loss: 2.5353
Epoch [1/10], Learning Rate: 0.0001
Epoch [2/10], Train Accuracy: 10.4 %, Loss: 2.9400
Epoch [2/10], Learning Rate: 0.001
Epoch [3/10], Train Accuracy: 7.9 %, Loss: 2.8676
Epoch [3/10], Learning Rate: 0.0009779754323328191
Epoch [4/10], Train Accuracy: 14.6 %, Loss: 2.1394
Epoch [4/10], Learning Rate: 0.0009140576474687263
Epoch [5/10], Train Accuracy: 18.3 %, Loss: 2.2149
Epoch [5/10], Learning Rate: 0.000814503363531613
Epoch [6/10], Train Accuracy: 22.8 %, Loss: 1.9537
Epoch [6/10], Learning Rate: 0.0006890576474687264
Epoch [7/10], Train Accuracy: 26.8 %, Loss: 1.8089
Epoch [7/10], Learning Rate: 0.00055
Epoch [8/10], Train Accuracy: 33.3 %, Loss: 2.3255
Epoch [8/10], Learning Rate: 0.0004109423525312737
Epoch [9/10], Train Accuracy: 35.2 %, Loss: 1.6354
Epoch [9/10], Learning Rate: 0.00028549663646838715
Epoch [10/10], Train Accuracy: 42.6 %, Loss: 1.5635
Epoch [10/10], Learning Rate: 0.00018594235253127368


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 42.0 %


In [32]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（25 epochs）

In [15]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 25
# warm-upするepoch数を設定
warmup_epochs = 3

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet34_HandSigns_25epochs_CLRS_restest.pth')

Epoch [1/25], Train Accuracy: 12.2 %, Loss: 2.7856
Epoch [1/25], Learning Rate: 0.0001
Epoch [2/25], Train Accuracy: 12.6 %, Loss: 2.7225
Epoch [2/25], Learning Rate: 0.00039999999999999996
Epoch [3/25], Train Accuracy: 12.9 %, Loss: 2.7140
Epoch [3/25], Learning Rate: 0.0007
Epoch [4/25], Train Accuracy: 14.5 %, Loss: 2.3288
Epoch [4/25], Learning Rate: 0.001
Epoch [5/25], Train Accuracy: 15.1 %, Loss: 2.0655
Epoch [5/25], Learning Rate: 0.000996451615591515
Epoch [6/25], Train Accuracy: 20.7 %, Loss: 1.6467
Epoch [6/25], Learning Rate: 0.000985862422507884
Epoch [7/25], Train Accuracy: 26.0 %, Loss: 2.4087
Epoch [7/25], Learning Rate: 0.0009683994186497131
Epoch [8/25], Train Accuracy: 25.8 %, Loss: 1.6453
Epoch [8/25], Learning Rate: 0.0009443380060197386
Epoch [9/25], Train Accuracy: 29.4 %, Loss: 1.7985
Epoch [9/25], Learning Rate: 0.0009140576474687263
Epoch [10/25], Train Accuracy: 33.9 %, Loss: 1.6638
Epoch [10/25], Learning Rate: 0.0008780358823396353
Epoch [11/25], Train Accu

In [16]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 84.0 %


In [17]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（50 epochs）

In [18]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet34_HandSigns_50epochs_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.1 %, Loss: 2.5196
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 12.7 %, Loss: 2.4703
Epoch [2/50], Learning Rate: 0.00028
Epoch [3/50], Train Accuracy: 10.7 %, Loss: 2.1969
Epoch [3/50], Learning Rate: 0.00045999999999999996
Epoch [4/50], Train Accuracy: 12.9 %, Loss: 2.2206
Epoch [4/50], Learning Rate: 0.0006399999999999999
Epoch [5/50], Train Accuracy: 16.3 %, Loss: 2.0873
Epoch [5/50], Learning Rate: 0.00082
Epoch [6/50], Train Accuracy: 19.8 %, Loss: 1.8432
Epoch [6/50], Learning Rate: 0.001
Epoch [7/50], Train Accuracy: 22.7 %, Loss: 2.2371
Epoch [7/50], Learning Rate: 0.0009991120277927223
Epoch [8/50], Train Accuracy: 27.1 %, Loss: 1.9560
Epoch [8/50], Learning Rate: 0.000996451615591515
Epoch [9/50], Train Accuracy: 31.4 %, Loss: 1.2886
Epoch [9/50], Learning Rate: 0.00099202926282791
Epoch [10/50], Train Accuracy: 34.2 %, Loss: 1.5177
Epoch [10/50], Learning Rate: 0.000985862422507884
Epoch [11/50], Train Accuracy: 37.4 %, L

In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 99.0 %


In [20]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（75 epochs）

In [24]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 75
# warm-upするepoch数を設定
warmup_epochs = 8

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet34_HandSigns_75epochs_CLRS_restest.pth')

Epoch [1/75], Train Accuracy: 9.4 %, Loss: 2.4445
Epoch [1/75], Learning Rate: 0.0001
Epoch [2/75], Train Accuracy: 11.6 %, Loss: 2.5369
Epoch [2/75], Learning Rate: 0.00021250000000000002
Epoch [3/75], Train Accuracy: 14.0 %, Loss: 2.6014
Epoch [3/75], Learning Rate: 0.000325
Epoch [4/75], Train Accuracy: 12.9 %, Loss: 2.1872
Epoch [4/75], Learning Rate: 0.0004375
Epoch [5/75], Train Accuracy: 13.0 %, Loss: 2.3534
Epoch [5/75], Learning Rate: 0.00055
Epoch [6/75], Train Accuracy: 19.4 %, Loss: 2.2033
Epoch [6/75], Learning Rate: 0.0006625
Epoch [7/75], Train Accuracy: 24.0 %, Loss: 2.4295
Epoch [7/75], Learning Rate: 0.0007750000000000001
Epoch [8/75], Train Accuracy: 23.4 %, Loss: 1.5270
Epoch [8/75], Learning Rate: 0.0008875
Epoch [9/75], Train Accuracy: 27.9 %, Loss: 2.1890
Epoch [9/75], Learning Rate: 0.001
Epoch [10/75], Train Accuracy: 32.4 %, Loss: 1.7843
Epoch [10/75], Learning Rate: 0.0009996052735444863
Epoch [11/75], Train Accuracy: 33.1 %, Loss: 1.5528
Epoch [11/75], Learn

In [25]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 98.0 %


In [26]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（100 epochs）

In [27]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet34_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet34_HandSigns_100epochs_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.7 %, Loss: 2.6536
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 15.5 %, Loss: 2.5740
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 12.8 %, Loss: 2.8143
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 12.7 %, Loss: 2.5762
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 14.3 %, Loss: 2.1224
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 19.4 %, Loss: 2.0640
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 21.8 %, Loss: 2.4305
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 23.3 %, Loss: 2.0600
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 30.0 %, Loss: 2.2265
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 33.3 %, Loss: 1.1722
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 31.8 %, Loss: 1.5052
Epo

In [28]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 99.0 %


In [29]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()